In [5]:
# https://huggingface.co/Dorjzodovsuren/mongolian-gpt2/tree/main
from transformers import AutoTokenizer, AutoModelForTokenClassification

tokenizer = AutoTokenizer.from_pretrained("Dakie/mongolian-xlm-roberta-base")
model = AutoModelForTokenClassification.from_pretrained("Dakie/mongolian-xlm-roberta-base")

# prepare input
text = "Replace me by any text you'd like."
encoded_input = tokenizer(text, return_tensors='pt')

# forward pass
output = model(**encoded_input)

tokenizer_config.json:   0%|          | 0.00/446 [00:00<?, ?B/s]

c:\Users\quezi\.conda\envs\weibodatacleaning\lib\site-packages\huggingface_hub\file_download.py:148: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\quezi\.cache\huggingface\hub\models--Dakie--mongolian-xlm-roberta-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/934 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

TokenClassifierOutput(loss=None, logits=tensor([[[ 4.3820, -1.1966, -1.1492, -0.6705, -1.0283],
         [ 7.4520, -2.8143, -1.1108, -2.5972, -2.0574],
         [ 6.7952, -2.5650, -1.0128, -2.5681, -1.7309],
         [ 7.8652, -2.9758, -1.3628, -2.9932, -2.3137],
         [ 7.2601, -2.5878, -0.9643, -3.1089, -1.9400],
         [ 8.0008, -3.0485, -1.0182, -2.7168, -2.2959],
         [ 7.6846, -2.8013, -0.7317, -3.0018, -2.3196],
         [ 8.5289, -2.7921, -1.9424, -2.8829, -2.3142],
         [ 8.4706, -2.9610, -1.7158, -2.9777, -2.3410],
         [ 8.2558, -2.7118, -1.8264, -2.6290, -2.0615],
         [ 8.1811, -2.6986, -1.6056, -2.7901, -2.1689],
         [ 8.2630, -3.0932, -1.7490, -3.0840, -1.8840],
         [ 4.8085, -1.1370, -1.4784, -0.7399, -0.5023]]],
       grad_fn=<ViewBackward0>), hidden_states=None, attentions=None)

In [9]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("bayartsogt/mongolian-gpt2")
model = AutoModelForCausalLM.from_pretrained("bayartsogt/mongolian-gpt2")

tokenizer_config.json:   0%|          | 0.00/207 [00:00<?, ?B/s]

c:\Users\quezi\.conda\envs\weibodatacleaning\lib\site-packages\huggingface_hub\file_download.py:148: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\quezi\.cache\huggingface\hub\models--bayartsogt--mongolian-gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.json:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.33M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.10M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/864 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/510M [00:00<?, ?B/s]

In [10]:
# 准备输入文本，这是模型生成续写的起点
prompt_text = "Өглөөний нар мандаж,"  # 假设这意味着“早晨的太阳升起，”

# 将文本编码为模型可以理解的格式
input_ids = tokenizer.encode(prompt_text, return_tensors='pt')

# 使用模型生成文本
# num_return_sequences=1 表示生成一个续写
# max_length 指定生成文本的最大长度，包括输入文本的长度
output = model.generate(input_ids, max_length=50, num_return_sequences=1)

# 解码生成的文本
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

print(generated_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Өглөөний нар мандаж, маргааш ирэх нь тодорхой байсан тул Эзэн Богд Чингис хааны хөшөөний өмнө зассан арван хоёр бугат асуудал дэгджээ.
1911 оны 12 сарын 29-нд Манжийн хаан зарлиг гаргаж, Богд Хаант Монгол Улсын Засгийн газар нь Монгол Улсын тусгаар тогтнолыг яаралтай
